In [6]:
import numpy as np
#if this fails, you need to put the case_studies.py file in the same folder
#from case_studies import *
from case_studies import *
from scipy.optimize import minimize
import time

In [7]:
def backtracking_line_search(f, grad_f, x, p, alpha_init=1.0, c1=1e-4, rho=0.9):
    """
    Performs backtracking line search to find a suitable step size.
    
    Parameters:
        f: Function to minimize.
        grad_f: Gradient of f.
        x: Current point.
        p: Search direction.
        alpha_init: Initial step size.
        c1: Armijo condition parameter.
        rho: Reduction factor for step size.
    
    Returns:
        alpha: Suitable step size.
    """
    alpha = alpha_init
    while f(x + alpha * p) > f(x) + c1 * alpha * np.dot(grad_f(x), p):
        alpha *= rho
    return alpha

## Steepest Descent

In [8]:
#Steepest Descent

def steepest_descent(f, grad_f, x0, c1=1e-4, rho=0.9, tol=1e-6, max_iters=1000):
    """
    Implements the Steepest Descent Algorithm with Backtracking Line Search.
    
    Parameters:
        f: Function to minimize.
        grad_f: Gradient of f.
        x0: Initial point.
        c1: Armijo condition parameter.
        rho: Reduction factor for step size.
        tol: Tolerance for stopping condition.
        max_iters: Maximum number of iterations.
    
    Returns:
        x: Estimated minimum.
        history: List of iterates for analysis.
    """
    x = x0
    beta = 1.0  # Initial beta value
    history = [x0]
    
    for k in range(max_iters):
        p = -grad_f(x)  # Steepest descent direction
        if np.linalg.norm(p) < tol:
            break  # Stopping condition
        
        alpha = backtracking_line_search(f, grad_f, x, p, beta, c1, rho)
        x = x + alpha * p
        beta = alpha / rho  # Update beta
        history.append(x)
    
    return x, history


## Newton's Algorithm

In [9]:
#Newton's Algorithm

def newtons_method(f, grad_f, hessian_f, x0, c1=1e-4, rho=0.9, tol=1e-6, max_iters=1000):
    """
    Implements Newton's Method with a modified Hessian when necessary.
    
    Parameters:
        f: Function to minimize.
        grad_f: Gradient of f.
        hessian_f: Hessian (second derivative) of f.
        x0: Initial point.
        c1: Armijo condition parameter.
        rho: Reduction factor for step size.
        tol: Tolerance for stopping condition.
        max_iters: Maximum number of iterations.
    
    Returns:
        x: Estimated minimum.
        history: List of iterates for analysis.
    """
    x = x0
    history = [x0]
    
    for k in range(max_iters):
        grad = grad_f(x)
        hessian = hessian_f(x)
        
        if np.linalg.norm(grad) < tol:
            break  # Stopping condition
        
        if np.all(np.linalg.eigvals(hessian) > 0):
            p = -np.linalg.solve(hessian, grad)  # Newton's direction
        else:
            eigvals, eigvecs = np.linalg.eigh(hessian)
            H = sum((1 / abs(eigval)) * np.outer(eigvec, eigvec) for eigval, eigvec in zip(eigvals, eigvecs))
            p = -H @ grad  # Modified direction
        
        alpha = backtracking_line_search(f, grad_f, x, p, 1.0, c1, rho)
        x = x + alpha * p
        history.append(x)
    
    return x, history

In [11]:

# List of functions, gradients, and Hessians
functions = [f1, f2, f3, f4, f5]
dfs = [df1, df2, df3, df4, df5]
Hfs = [Hf1, Hf2, Hf3, Hf4, Hf5]
function_names = ["f1", "f2", "f3", "f4", "f5"]

# Function to run benchmark
def benchmark_algorithm(algorithm, f, grad_f, hessian_f, x0, x_opt, algo_name):
    start_time = time.time()
    
    if algo_name == "Newton":
        x_min, history = newtons_method(f, grad_f, hessian_f, x0)
    else:
        x_min, history = steepest_descent(f, grad_f, x0)

    end_time = time.time()
    
    # Compute performance metrics
    num_iterations = len(history)
    final_value = f(x_min)
    error = np.linalg.norm(x_min - x_opt)
    time_taken = end_time - start_time

    return num_iterations, final_value, error, time_taken

# Run benchmarks
results = []

for i in range(len(functions)):
    f, df, Hf = functions[i], dfs[i], Hfs[i]
    x0 = np.random.randn(2)  # Initialize x0 randomly in 2D space   ### THIS IS FOR ROSENBROCK FUNCTION AS ITS MAXXED AT 2D
    x_opt_val = x_opt(function_names[i], len(x0))

    # Benchmark Steepest Descent
    sd_metrics = benchmark_algorithm(steepest_descent, f, df, Hf, x0, x_opt_val, "Steepest Descent")

    # Benchmark Newton's Method
    newton_metrics = benchmark_algorithm(newtons_method, f, df, Hf, x0, x_opt_val, "Newton")

    # Store results
    results.append((function_names[i], *sd_metrics, *newton_metrics))

# Print results
print("\nBenchmark Results:")
print("Function | SD Iter | SD f(x) | SD Error | SD Time | Newton Iter | Newton f(x) | Newton Error | Newton Time")
for r in results:
    print(f"{r[0]:^8} | {r[1]:^7} | {r[2]:^7.4f} | {r[3]:^8.4f} | {r[4]:^7.4f} | {r[5]:^11} | {r[6]:^8.4f} | {r[7]:^10.4f} | {r[8]:^8.4f}")



Benchmark Results:
Function | SD Iter | SD f(x) | SD Error | SD Time | Newton Iter | Newton f(x) | Newton Error | Newton Time
   f1    |  1001   | 0.0049  |  0.0697  | 0.2784  |      2      |  0.0000  |   0.0000   |  0.0040 
   f2    |  1001   | 0.0045  |  0.1492  | 0.0380  |     14      |  0.0000  |   0.0000   |  0.0022 
   f3    |  1001   | 3.5520  |  0.0577  | 0.3085  |      6      |  0.0000  |   0.0000   |  0.0052 
   f4    |   113   | 0.0000  |  0.0000  | 0.0516  |      8      |  0.0000  |   0.0000   |  0.0042 
   f5    |  1001   | 0.0000  |  0.0110  | 0.1958  |     12      |  0.0000  |   0.0061   |  0.0051 
